In [112]:
import pandas as pd
column_names = ['Item #', 'Vendor Item #', 'Job Bag #', 'Description', 'Dia Qlty', 'Metal','Size','Qty.','Weight','Cost','Amount', 'Extra1', 'Extra2', 'Extra3', 'Sizereal', 'Extra5', 'Extra6']
df = pd.read_excel(r'C:\Users\Admin\Downloads\VIMO_PO.xlsx', skiprows=8,header=None) ## Read the Excel file starting from row 11 (skip first 10 rows)

df.columns = column_names
df.head(10)



,Item #,Vendor Item #,Job Bag #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,NaN,NaN,WR3125FGMC- YGC2\nZR5044ZMC- WGC4,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grand Total: $0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
import pandas as pd

# Step 1: Read Excel
column_names = ['Item #', 'Vendor Item #', 'Job Bag #', 'Description', 'Dia Qlty', 'Metal',
                'Size', 'Qty.', 'Weight', 'Cost', 'Amount', 'Extra1', 'Extra2', 'Extra3',
                'Extra4', 'Extra5', 'Extra6']

df = pd.read_excel(r'C:\Users\Admin\Downloads\VIMO_PO.xlsx', skiprows=8, header=None)
df.columns = column_names

# Step 2: Identify rows where 'Item #' has multiple entries (contains '\n')
mask = df['Item #'].astype(str).str.contains('\n', na=False)

# Step 3: Expand rows
df_expanded = df[mask].copy()
df_expanded['Item #'] = df_expanded['Item #'].str.split('\n')
df_expanded = df_expanded.explode('Item #', ignore_index=True)

# Step 4: Keep rows without '\n' and combine with expanded ones
df_single = df[~mask].copy()
df_cleaned = pd.concat([df_single, df_expanded], ignore_index=True)

# Step 5: Optional cleanup — remove extra whitespace
df_cleaned['Item #'] = df_cleaned['Item #'].str.strip()

# ✅ Step 6: Drop NaNs from 'Item #' column
df_cleaned = df_cleaned[df_cleaned['Item #'].notna()]

# Reset index
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned = df_cleaned.drop(index=0).reset_index(drop=True)

# Show result
df_cleaned[['Item #']].head(10)


,Item #
0,1 SPO-3999YD
1,2 SPO-4000WC


In [151]:
df.head(10)
# Step 1: Drop the original 'Vendor Item #' column
df = df.drop(columns=['Vendor Item #'])

# Step 2: Rename 'Job Bag #' to 'Vendor Item #'
df = df.rename(columns={'Job Bag #': 'Vendor Item #'})

# Step 3: Identify rows with multiple 'Vendor Item #' entries (split by '\n')
mask_vendor = df['Vendor Item #'].astype(str).str.contains('\n', na=False)

# Step 4: Expand rows with '\n' into separate rows
df_expanded = df[mask_vendor].copy()
df_expanded['Vendor Item #'] = df_expanded['Vendor Item #'].str.split('\n')
df_expanded = df_expanded.explode('Vendor Item #', ignore_index=True)

# Step 5: Keep rows without '\n' and combine with expanded
df_single = df[~mask_vendor].copy()
df_cleaned = pd.concat([df_single, df_expanded], ignore_index=True)

# Step 6: Clean up whitespace and drop empty/null values
df_cleaned['Vendor Item #'] = df_cleaned['Vendor Item #'].str.strip()
df_cleaned = df_cleaned[df_cleaned['Vendor Item #'].notna() & (df_cleaned['Vendor Item #'] != '')].reset_index(drop=True)

# ✅ Preview the cleaned column
df_cleaned[['Vendor Item #']].head(10)


KeyError: "['Vendor Item #'] not found in axis"

In [82]:
import pandas as pd
import re

# Regex pattern to split CTW blocks (assumes starts with 0.xxCTW and follows with a space)
ctw_split_pattern = r'(?=\b0\.\d{2}CTW)'

# Process the dataframe
new_rows = []

for i, row in df.iterrows():
    size_val = str(row['Size']).strip()

    # Check if multiple CTW entries are merged in one cell
    if re.findall(r'0\.\d{2}CTW', size_val):
        # Split into chunks at each new CTW start
        parts = re.split(ctw_split_pattern, size_val)
        parts = [p.strip() for p in parts if p.strip()]  # clean empty

        # Create a new row for each CTW part
        for part in parts:
            new_row = row.copy()
            new_row['Size'] = part
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Drop rows with NaN in Size column and reset index
df_cleaned = df_cleaned.dropna(subset=['Size']).reset_index(drop=True)

# Preview result
df_cleaned[['Size']].head(10) ## Description


,Size
0,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...
1,0.40CTW DIA BAND 14KW R6174-040


In [83]:
df.head()

,Item #,Vendor Item #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Extra4,Extra5,Extra6
0,NaN,WR3125FGMC- YGC2\nZR5044ZMC- WGC4,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grand Total: $0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
import pandas as pd
import numpy as np

# Define quality keywords that may appear in Dia Qlty
quality_keywords = ['A', 'AA', 'VS-G', 'I1I2IJ', 'I2-I2-GH', 'I2I3- TTLB', 'I2I3(GH) White']

# Initialize list for new rows
new_rows = []

for _, row in df.iterrows():
    amount_val = str(row['Amount'])

    if any(q in amount_val for q in quality_keywords):
        # Split on newline if exists
        parts = amount_val.strip().split('\n')
        
        for part in parts:
            tokens = part.split()
            if len(tokens) >= 2:
                dia_qlty = tokens[0]
                metal = tokens[1]
            elif len(tokens) == 1:
                dia_qlty = tokens[0]
                metal = np.nan
            else:
                continue
            
            new_row = row.copy()
            new_row['Dia Qlty'] = dia_qlty
            new_row['Metal'] = metal
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Create new DataFrame
df_updated = pd.DataFrame(new_rows)

# Optional cleanup
df_updated = df_updated.reset_index(drop=True)

# Display result
df_updated[['Dia Qlty', 'Metal']].dropna().head(10)


,Dia Qlty,Metal
1,A,18KY
2,AA,14KW


In [115]:
df.head()

,Item #,Vendor Item #,Job Bag #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,NaN,NaN,WR3125FGMC- YGC2\nZR5044ZMC- WGC4,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grand Total: $0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
df['Sizereal'].head(10)

0                                          NaN
1    9                1\n4.5              1\n2
2                                          NaN
3                                          NaN
4                                          NaN
Name: Sizereal, dtype: object

In [117]:
import pandas as pd

# Assume df is your DataFrame

# Step 1: Ensure column names are clean
df.columns = df.columns.str.strip()

# Step 2: Work only on rows where Sizereal has '\n'
mask = df['Sizereal'].astype(str).str.contains('\n', na=False)
df_split = df[mask].copy()

# Step 3: Split those rows by newline and explode into multiple rows
df_split['Sizereal'] = df_split['Sizereal'].str.split('\n')
df_split = df_split.explode('Sizereal', ignore_index=True)

# Step 4: Strip extra spaces
df_split['Sizereal'] = df_split['Sizereal'].str.strip()

# Step 5: Get remaining rows (those without newline in Sizereal)
df_rest = df[~mask].copy()

# Step 6: Combine both and reset index
df_cleaned = pd.concat([df_rest, df_split], ignore_index=True)
df_cleaned = df_cleaned[df_cleaned['Sizereal'].notna()].reset_index(drop=True)

# ✅ Output preview
df_cleaned[['Sizereal']].head(10)


,Sizereal
0,9 1
1,4.5 1
2,2


In [136]:
import pandas as pd
import numpy as np
import re
column_names = ['Item #', 'Vendor Item #', 'Job Bag #', 'Description', 'Dia Qlty', 'Metal','Size','Qty.','Weight','Cost','Amount', 'Extra1', 'Extra2', 'Extra3', 'Sizereal', 'Extra5', 'Extra6']
df = pd.read_excel(r'C:\Users\Admin\Downloads\VIMO_PO.xlsx', skiprows=8,header=None) ## Read the Excel file starting from row 11 (skip first 10 rows)

df.columns = column_names
#df.head(10)


# Step 1: Read Excel
column_names = ['Item #', 'Vendor Item #', 'Job Bag #', 'Description', 'Dia Qlty', 'Metal',
                'Size', 'Qty.', 'Weight', 'Cost', 'Amount', 'Extra1', 'Extra2', 'Extra3',
                'Sizereal', 'Extra5', 'Extra6']

df = pd.read_excel(r'C:\Users\Admin\Downloads\VIMO_PO.xlsx', skiprows=8, header=None)
df.columns = column_names

# Step 2: Identify rows where 'Item #' has multiple entries (contains '\n')
mask1 = df['Item #'].astype(str).str.contains('\n', na=False)

# Step 3: Expand rows
df_expanded1 = df[mask].copy()
df_expanded1['Item #'] = df_expanded1['Item #'].str.split('\n')
df_expanded1 = df_expanded1.explode('Item #', ignore_index=True)

# Step 4: Keep rows without '\n' and combine with expanded ones
df_single1 = df[~mask1].copy()
df_cleaned1 = pd.concat([df_single1, df_expanded1], ignore_index=True)

# Step 5: Optional cleanup — remove extra whitespace
df_cleaned1['Item #'] = df_cleaned1['Item #'].str.strip()

# ✅ Step 6: Drop NaNs from 'Item #' column
df_cleaned1 = df_cleaned1[df_cleaned1['Item #'].notna()]

# Reset index
df_cleaned1.reset_index(drop=True, inplace=True)
df_cleaned1 = df_cleaned1.drop(index=0).reset_index(drop=True)

# Show result
df_cleaned1[['Item #']].head(10)



# Step 1: Drop the original 'Vendor Item #' column
df = df.drop(columns=['Vendor Item #'])

# Step 2: Rename 'Job Bag #' to 'Vendor Item #'
df = df.rename(columns={'Job Bag #': 'Vendor Item #'})

# Step 3: Identify rows with multiple 'Vendor Item #' entries (split by '\n')
mask_vendor = df['Vendor Item #'].astype(str).str.contains('\n', na=False)

# Step 4: Expand rows with '\n' into separate rows
df_expanded2 = df[mask_vendor].copy()
df_expanded2['Vendor Item #'] = df_expanded2['Vendor Item #'].str.split('\n')
df_expanded2 = df_expanded2.explode('Vendor Item #', ignore_index=True)

# Step 5: Keep rows without '\n' and combine with expanded
df_single2 = df[~mask_vendor].copy()
df_cleaned2 = pd.concat([df_single2, df_expanded2], ignore_index=True)

# Step 6: Clean up whitespace and drop empty/null values
df_cleaned2['Vendor Item #'] = df_cleaned2['Vendor Item #'].str.strip()
df_cleaned2 = df_cleaned2[df_cleaned2['Vendor Item #'].notna() & (df_cleaned2['Vendor Item #'] != '')].reset_index(drop=True)

# ✅ Preview the cleaned column
df_cleaned2[['Vendor Item #']].head(10)




# Regex pattern to split CTW blocks (assumes starts with 0.xxCTW and follows with a space)
ctw_split_pattern = r'(?=\b0\.\d{2}CTW)'

# Process the dataframe
new_rows = []

for i, row in df.iterrows():
    size_val = str(row['Size']).strip()

    # Check if multiple CTW entries are merged in one cell
    if re.findall(r'0\.\d{2}CTW', size_val):
        # Split into chunks at each new CTW start
        parts = re.split(ctw_split_pattern, size_val)
        parts = [p.strip() for p in parts if p.strip()]  # clean empty

        # Create a new row for each CTW part
        for part in parts:
            new_row = row.copy()
            new_row['Size'] = part
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Drop rows with NaN in Size column and reset index
df_cleaned2 = df_cleaned2.dropna(subset=['Size']).reset_index(drop=True)

# Preview result
df_cleaned2[['Size']].head(10) ## Description






# Define quality keywords that may appear in Dia Qlty
quality_keywords = ['A', 'AA', 'VS-G', 'I1I2IJ', 'I2-I2-GH', 'I2I3- TTLB', 'I2I3(GH) White']

# Initialize list for new rows
new_rows = []

for _, row in df.iterrows():
    amount_val = str(row['Amount'])

    if any(q in amount_val for q in quality_keywords):
        # Split on newline if exists
        parts = amount_val.strip().split('\n')
        
        for part in parts:
            tokens = part.split()
            if len(tokens) >= 2:
                dia_qlty = tokens[0]
                metal = tokens[1]
            elif len(tokens) == 1:
                dia_qlty = tokens[0]
                metal = np.nan
            else:
                continue
            
            new_row = row.copy()
            new_row['Dia Qlty'] = dia_qlty
            new_row['Metal'] = metal
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Create new DataFrame
df_updated2 = pd.DataFrame(new_rows)

# Optional cleanup
df_updated2 = df_updated2.reset_index(drop=True)

# Display result
df_updated2[['Dia Qlty', 'Metal']].dropna().head(10)



df.columns = df.columns.str.strip()

# Step 2: Work only on rows where Sizereal has '\n'
mask3 = df['Sizereal'].astype(str).str.contains('\n', na=False)
df_split3 = df[mask3].copy()

# Step 3: Split those rows by newline and explode into multiple rows
df_split3['Sizereal'] = df_split3['Sizereal'].str.split('\n')
df_split3 = df_split3.explode('Sizereal', ignore_index=True)

# Step 4: Strip extra spaces
df_split3['Sizereal'] = df_split3['Sizereal'].str.strip()

# Step 5: Get remaining rows (those without newline in Sizereal)
df_rest3 = df[~mask3].copy()

# Step 6: Combine both and reset index
df_cleaned3 = pd.concat([df_rest3, df_split3], ignore_index=True)
df_cleaned3 = df_cleaned3[df_cleaned3['Sizereal'].notna()].reset_index(drop=True)

# ✅ Output preview
df_cleaned3[['Sizereal']].head(10)




,Sizereal
0,9 1
1,4.5 1
2,2


In [141]:
df_cleaned1.head()

,Item #,Vendor Item #,Job Bag #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,1 SPO-3999YD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
1,2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...


In [142]:
df_cleaned2.head()

,Item #,Vendor Item #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,NaN,WR3125FGMC- YGC2,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ZR5044ZMC- WGC4,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
df_updated2.head()

,Item #,Vendor Item #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,NaN,WR3125FGMC- YGC2\nZR5044ZMC- WGC4,NaN,NaN,NaN,0.50CTW HIGH POLISH DIA GENTS BAND 18KY R9000-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,A,18KY,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
2,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,AA,14KW,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1\n4.5 1\n2,NaN,$0.00 $0.00\n$0.00 ...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
df_cleaned3.head()

,Item #,Vendor Item #,Description,Dia Qlty,Metal,Size,Qty.,Weight,Cost,Amount,Extra1,Extra2,Extra3,Sizereal,Extra5,Extra6
0,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,9 1,NaN,$0.00 $0.00\n$0.00 ...
1,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,4.5 1,NaN,$0.00 $0.00\n$0.00 ...
2,1 SPO-3999YD\n2 SPO-4000WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A 18KY\nAA 14KW,NaN,NaN,NaN,2,NaN,$0.00 $0.00\n$0.00 ...


In [148]:
df_cleaned1 = df_cleaned1[['Item #']].copy()
df_cleaned2 =  df_cleaned2[['Vendor Item #','Size']].copy()
#df_updated2 =  df_updated2[['Dia Qlty','Metal']].copy()
df_updated2 = df_updated2[['Dia Qlty', 'Metal']].dropna().reset_index(drop=True)
df_cleaned3 = df_cleaned3[['Sizereal']].copy()
df_final = pd.concat([df_cleaned1, df_cleaned2, df_updated2, df_cleaned3], axis=1)
df_final.head()

KeyError: "['Size'] not in index"